This notebook shows how to forecast stock prices using machine learning and neural networks. In particular I chose GOOGL stock (Alphabet) and I will be using LSTM RNNs to predict future prices.

First download the dataset from the following link:
https://www.kaggle.com/datasets/szrlee/stock-time-series-20050101-to-20171231?resource=download&select=GOOGL_2006-01-01_to_2018-01-01.csv

In [1]:
import pandas as pd

In [2]:
stock_df = pd.read_csv("GOOGL_2006-01-01_to_2018-01-01.csv")
print(stock_df.head())

         Date    Open    High     Low   Close    Volume   Name
0  2006-01-03  211.47  218.05  209.32  217.83  13137450  GOOGL
1  2006-01-04  222.17  224.70  220.09  222.84  15292353  GOOGL
2  2006-01-05  223.22  226.00  220.97  225.85  10815661  GOOGL
3  2006-01-06  228.66  235.49  226.85  233.06  17759521  GOOGL
4  2006-01-09  233.44  236.94  230.70  233.68  12795837  GOOGL
